In [4]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [5]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data2_smpl30_2/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['label'], axis=1)
        self.Y = df['label']
        self.dt=data
class uji2(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 

dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
#dtt =dtuji('tambahan') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt


In [6]:
dtg =dtuji('gabung') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt

In [7]:
#pd.read_csv("../data2_smpl30_2/latih_sampled30_1.csv", delimiter=',').tail()

In [8]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data2_smpl30_2/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','0.1'], axis=1)
        self.Y = df['0.1']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 
df1= dtlatih('latih_sampled30_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('latih_sampled30_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('latih_sampled30_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('latih_sampled30_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('latih_sampled30_5') #df5.X,df5.Y,df5.dt

/home/biofarmaka/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
du5=dtlatih('uji_sampled30_5') #df5.X,df5.Y,df5.dt
du4=dtlatih('uji_sampled30_4') #df5.X,df5.Y,df5.dt
du3=dtlatih('uji_sampled30_3') #df5.X,df5.Y,df5.dt
du2=dtlatih('uji_sampled30_2') #df5.X,df5.Y,df5.dt
du1=dtlatih('uji_sampled30_1') #df5.X,df5.Y,df5.dt

In [12]:
#df.X

In [13]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


#RF

In [45]:
def rf(df):
    # Number of trees in random forest
   # n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2500, num = 3)]
    # Number of features to consider at every split
    #max_features = ['auto', None]
    # Maximum number of levels in tree
    #max_depth = [int(x) for x in np.linspace(10, 100, num = 2)]
    #max_depth.append(None)
    # Minimum number of samples required to split a node
    random_grid = {'n_estimators': [100,300,500,1000],
                   #'max_features' : [1281],
                   #'max_depth': max_depth,                 
                   #'bootstrap': [True,False],
                   'max_samples': [1000,5000,10000,15000,18000]
                   }
    rf = BaggingClassifier(bootstrap=True,base_estimator=DecisionTreeClassifier(class_weight='balanced'))
    #rf = RandomForestClassifier()
    
    
    rf = GridSearchCV(rf, random_grid, n_jobs=15, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    rf.fit(df.X, df.Y)
    filename = '../data2_smpl30_2/model/rf_{0}'.format(df.dt)
    pickle.dump(rf, open(filename, 'wb'))
    return rf

In [49]:
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_latih_sampled30_{1}'.format(model.__name__,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_{1}'.format(model.__name__,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print(rf.cv_results_['rank_test_AUC']-1)
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))

def prediksi2(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.X))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(data.Y.value_counts())
    print(confusion_matrix(data.Y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    #pred[2] = data.com
    #pred[3] = data.prot
    print('Accuracy:',accuracy_score(data.Y, prob_x[1]))
    print('f1:',f1_score(data.Y, prob_x[1]))
    print('precision:',precision_score(data.Y, prob_x[1]))
    print('recall:',recall_score(data.Y, prob_x[1]))
    print('roc_auc:',roc_auc_score(data.Y, prob_x[1]))
    print('Confusion matrix:')
    print(confusion_matrix(data.Y, prob_x[1]))
    print('Results on the train set:')
    print(classification_report(data.Y, prob_x[1]))
    #print(pred.head())
    #print(prob_x)
    #pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    #pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [42]:
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    rf(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi2(md,model,p,datauji3,d)
def fitn2pred(model,p,datalatih,datauji1,datauji2,d):
    rf(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
def fitnpred(model,p,datalatih,datauji1,d):
    rf(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)

In [46]:
#+tuning mtry
fitn3pred(rf,0.5,df1,dtf,dtl,du1,1)


model file : rf_1
Best parameters found: {'max_samples': 10000, 'n_estimators': 1000}
CV RESULT :
[11 10  9  8  6  7  4  5  3  1  2  0 12 13 14 15 16 17 18 19]
AUC : 0.999305295988367
Accuracy : 0.9860850846857826
F-measure : 0.9856195702337954
Precision : 0.9721700760016561
Recall : 0.9994572591587516
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[199  41]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[2387  661]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_sampled30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3460   77]
 [   0 3086]]


ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [47]:
md=report_rf(rf,df1,1)


model file : rf_1
Best parameters found: {'max_samples': 10000, 'n_estimators': 1000}
CV RESULT :
[11 10  9  8  6  7  4  5  3  1  2  0 12 13 14 15 16 17 18 19]
AUC : 0.999305295988367
Accuracy : 0.9860850846857826
F-measure : 0.9856195702337954
Precision : 0.9721700760016561
Recall : 0.9994572591587516


In [50]:
prediksi2(md,rf,0.5,du1,1)


threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_sampled30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3460   77]
 [   0 3086]]
Accuracy: 0.9883738487090442
f1: 0.9876780284845574
precision: 0.9756560227631995
recall: 1.0
roc_auc: 0.989115069267741
Confusion matrix:
[[3460   77]
 [   0 3086]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3537
           1       0.98      1.00      0.99      3086

    accuracy                           0.99      6623
   macro avg       0.99      0.99      0.99      6623
weighted avg       0.99      0.99      0.99      6623



In [51]:
prediksi(md,rf,0.5,dtf,1)
prediksi(md,rf,0.5,dtl,1)

threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[199  41]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[2387  661]
 [   0    0]]


In [12]:
#sebelum tuning max features aka mtry
fitn3pred(rf,0.5,df1,dtf,dtl,dtg,1)


model file : rf_1
Best parameters found: {'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9996789363248269
Accuracy : 0.9888103999371916
F-measure : 0.9883273007644942
Precision : 0.9771081390416558
Recall : 1.0
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[220  20]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[2598  450]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  gabung  : (0,1)
[[2742  466]
 [   0    0]]


KeyboardInterrupt: 

In [52]:
def rf(df):
    # Number of trees in random forest
   # n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2500, num = 3)]
    # Number of features to consider at every split
    #max_features = ['auto', None]
    # Maximum number of levels in tree
    #max_depth = [int(x) for x in np.linspace(10, 100, num = 2)]
    #max_depth.append(None)
    # Minimum number of samples required to split a node
    random_grid = {'n_estimators': [1000],
                   #'max_features' : max_features,
                   #'max_depth': max_depth,                 
                   #'bootstrap': [True,False],
                   'max_samples': [10000]
                   }
    rf = BaggingClassifier(bootstrap=True,base_estimator=DecisionTreeClassifier(class_weight='balanced'))
    #rf = RandomForestClassifier()
    
    
    rf = GridSearchCV(rf, random_grid, n_jobs=15, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    rf.fit(df.X, df.Y)
    filename = '../data2_smpl30_2/model/rf_{0}'.format(df.dt)
    pickle.dump(rf, open(filename, 'wb'))
    return rf

In [54]:
fitn3pred(rf,0.5,df2,dtf,dtl,du2,2)
fitn3pred(rf,0.5,df3,dtf,dtl,du3,3)
fitn3pred(rf,0.5,df4,dtf,dtl,du4,4)
fitn3pred(rf,0.5,df5,dtf,dtl,du5,5)

model file : rf_2
Best parameters found: {'max_samples': 10000, 'n_estimators': 1000}
CV RESULT :
[0]
AUC : 0.999464689786525
Accuracy : 0.9868617418353105
F-measure : 0.986259724384189
Precision : 0.9728996550464757
Recall : 1.0
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbafika  : (0,1)
[[177  63]
 [  0   0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbalinda  : (0,1)
[[2255  793]
 [   0    0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  uji_sampled30_2  : (0,1)
0    3450
1    3173
Name: 0.1, dtype: int64
[[3372   78]
 [   0 3173]]
Accuracy: 0.9882228597312396
f1: 0.9878580323785803
precision: 0.9760073823438942
recall: 1.0
roc_auc: 0.988695652173913
Confusion matrix:
[[3372   78]
 [   0 3173]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3450
           1       0.98      1.00      0.99      3173

    accuracy                           0.99      6623
   macro avg       0.99   

model file : rf_3_1
Best parameters found: {'max_features': 500, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9997517661381023
Accuracy : 0.9916644507515212
F-measure : 0.9912786709698475
Precision : 0.982709500620612
Recall : 1.0


GridSearchCV(cv=5,
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced')),
             n_jobs=15,
             param_grid={'max_features': [300, 500, 700, 900, 1100, 1281],
                         'max_samples': [15000], 'n_estimators': [500]},
             refit='AUC', return_train_score=True,
             scoring={'AUC': 'roc_auc', 'Accuracy': 'accuracy',
                      'F-measure': 'f1', 'Precision': 'precision',
                      'Recall': 'recall'})

In [38]:
report_rf(rf,df2,2)

model file : rf_3_2
Best parameters found: {'max_features': 700, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9999391511172779
Accuracy : 0.992072192822077
F-measure : 0.9917036994502604
Precision : 0.983549129149934
Recall : 1.0


GridSearchCV(cv=5,
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced')),
             n_jobs=15,
             param_grid={'max_features': [300, 500, 700, 900, 1100, 1281],
                         'max_samples': [15000], 'n_estimators': [500]},
             refit='AUC', return_train_score=True,
             scoring={'AUC': 'roc_auc', 'Accuracy': 'accuracy',
                      'F-measure': 'f1', 'Precision': 'precision',
                      'Recall': 'recall'})

In [40]:
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    rf1(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi(md,model,p,datauji3,d)

In [41]:
fitn3pred(rf,0.5,df2,dtf,dtl,dtg,2)
fitn3pred(rf,0.5,df3,dtf,dtl,dtg,3)
fitn3pred(rf,0.5,df4,dtf,dtl,dtg,4)
fitn3pred(rf,0.5,df5,dtf,dtl,dtg,5)

model file : rf_3_2
Best parameters found: {'max_features': 500, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9998965731483496
Accuracy : 0.9918457025186515
F-measure : 0.9914682542502018
Precision : 0.9830852535767767
Recall : 1.0
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbafika  : (0,1)
[[205  35]
 [  0   0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbalinda  : (0,1)
[[2483  565]
 [   0    0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  gabung  : (0,1)
[[2617  591]
 [   0    0]]
model file : rf_3_3
Best parameters found: {'max_features': 500, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9997452603035016
Accuracy : 0.989761733103925
F-measure : 0.9893114772662983
Precision : 0.9788547972819316
Recall : 1.0
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  mbafika  : (0,1)
[[240]]
threshold 0.5 
model_hasilsmote_3 
hasil prediksi  mbalinda  : (0,1)
[[2690  358]
 [   0    0]]
threshold 0.5 
model_hasilsmote_3 
hasil prediks

In [42]:
def rf2(df):
    # Number of trees in random forest
   # n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2500, num = 3)]
    # Number of features to consider at every split
    #max_features = ['auto', None]
    # Maximum number of levels in tree
    #max_depth = [int(x) for x in np.linspace(10, 100, num = 2)]
    #max_depth.append(None)
    # Minimum number of samples required to split a node
    random_grid = {'n_estimators': [500],
                   'max_features' : [1000],
                   #'max_depth': max_depth,                 
                   #'bootstrap': [True,False],
                   'max_samples': [15000]
                   }
    rf = BaggingClassifier(bootstrap=True,base_estimator=DecisionTreeClassifier(class_weight='balanced'))
    #rf = RandomForestClassifier()
    
    
    rf = GridSearchCV(rf, random_grid, n_jobs=15, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    rf.fit(df.X, df.Y)
    filename = '../data2_smpl30_2/model/rf_4_{0}'.format(df.dt)
    pickle.dump(rf, open(filename, 'wb'))
    return rf
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_4_latih_sampled30_{1}'.format(model.__name__,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_4_{1}'.format(model.__name__,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_4_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    rf2(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi(md,model,p,datauji3,d)
fitn3pred(rf,0.5,df1,dtf,dtl,dtg,1)

model file : rf_4_1
Best parameters found: {'max_features': 1000, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9997478542440114
Accuracy : 0.9898523846445547
F-measure : 0.9894031109825348
Precision : 0.9790306388446961
Recall : 1.0
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[227  13]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[2619  429]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  gabung  : (0,1)
[[2770  438]
 [   0    0]]


In [44]:
def rf3(df):
    # Number of trees in random forest
   # n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2500, num = 3)]
    # Number of features to consider at every split
    #max_features = ['auto', None]
    # Maximum number of levels in tree
    #max_depth = [int(x) for x in np.linspace(10, 100, num = 2)]
    #max_depth.append(None)
    # Minimum number of samples required to split a node
    random_grid = {'n_estimators': [500],
                   'max_features' : [1281],
                   #'max_depth': max_depth,                 
                   #'bootstrap': [True,False],
                   'max_samples': [15000]
                   }
    rf = BaggingClassifier(bootstrap=True,base_estimator=DecisionTreeClassifier(class_weight='balanced'))
    #rf = RandomForestClassifier()
    
    
    rf = GridSearchCV(rf, random_grid, n_jobs=15, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    rf.fit(df.X, df.Y)
    filename = '../data2_smpl30_2/model/rf_5_{0}'.format(df.dt)
    pickle.dump(rf, open(filename, 'wb'))
    return rf
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_5_latih_sampled30_{1}'.format(model.__name__,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_5_{1}'.format(model.__name__,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_5_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    rf3(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi(md,model,p,datauji3,d)
fitn3pred(rf,0.5,df1,dtf,dtl,dtg,1)

model file : rf_5_1
Best parameters found: {'max_features': 1281, 'max_samples': 15000, 'n_estimators': 500}
CV RESULT :
AUC : 0.9996002900023078
Accuracy : 0.9889009899008663
F-measure : 0.9884203935230247
Precision : 0.9771068599461754
Recall : 1.0
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[222  18]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[2616  432]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  gabung  : (0,1)
[[2762  446]
 [   0    0]]
